In [0]:
import pandas as pd
import numpy as np

# Spark에서 일괄 연산을 위한 함수 생성을 위해 pandas_udf 함수를 임포트
from pyspark.sql.functions import col, udf, pandas_udf, expr
from pyspark.sql.types import IntegerType, DoubleType, StringType, FloatType

# 사전 학습

## Python Lambda 함수 학습

In [0]:
# add라는 이름으로 Lambda 함수 정의
add = lambda x, y: x + y

# 여기에 인자를 주고 실제로 실행
print(add(3, 7)) 

10


In [0]:
# square라는 이름의 Lambda 함수 정의
square = lambda x: x ** 2

# 인자를 하나 주고 실행
print(square(5))  # Output: 25

25


In [0]:
# 만일 이 함수를 리스트의 각 원소에 적용하고 싶다면
# map이라는 함수를 사용 (이는 함수를 리스트의 원소에 적용해주는 역할을 수행)
numbers = [1, 2, 3, 4, 5]
squared_map = map(square, numbers)
print(squared_map)

In [0]:
# map은 iterator라는 것을 리턴해주며 이 내용을 보고 싶다면 list를 사용해서 리스트로 변환 가능
squared_list = list(squared_map)

In [0]:
squared_list

[1, 4, 9, 16, 25]

### 이런 기능을 Pandas DataFrame에도 적용하고 싶다면 사용 가능한 것이 Pandas의 apply 함수

## Spark에서 UDF 학습

Spark에는 Pandas의 apply와는 기능이 별도로 존재하지는 않고 기본적으로 컬럼 연산이나 UDF를 통해 비슷한 기능 구현이 가능하며 apply 함수에서처럼 열과 행의 구분이 딱히 있지 않음

In [0]:
data = {
    'id': [1, 2, 3],
    'name': ['luka doncici', 'lebron james', 'austin reaves'],
    'salary': [100, 200, 300]
}
sdf = spark.createDataFrame(pd.DataFrame(data))
sdf.show()

+---+-------------+------+
| id|         name|salary|
+---+-------------+------+
|  1| luka doncici|   100|
|  2| lebron james|   200|
|  3|austin reaves|   300|
+---+-------------+------+



In [0]:
# 예제 1: pyspark.sql.functions.udf: DataFrame에서만 사용 가능
import pyspark.sql.functions as F
from pyspark.sql.types import *

upperUDF = F.udf(lambda z:z.upper())  
sdf_v2 = sdf.withColumn("Capital Name", upperUDF("Name"))
sdf_v2.show()

+---+-------------+------+-------------+
| id|         name|salary| Capital Name|
+---+-------------+------+-------------+
|  1| luka doncici|   100| LUKA DONCICI|
|  2| lebron james|   200| LEBRON JAMES|
|  3|austin reaves|   300|AUSTIN REAVES|
+---+-------------+------+-------------+



In [0]:
sdf.withColumn("new salary", F.udf(lambda x: x*1.1)("salary")).show()

+---+-------------+------+------------------+
| id|         name|salary|        new salary|
+---+-------------+------+------------------+
|  1| luka doncici|   100|110.00000000000001|
|  2| lebron james|   200|220.00000000000003|
|  3|austin reaves|   300|             330.0|
+---+-------------+------+------------------+



In [0]:
# 예제 2: spark.udf.register: DataFrame 뿐만 아니라 Spark SQL에서도 사용가능
def upper(s):
   return s.upper()

# 먼저 SELECT를 통해 테스트
upperUDF = spark.udf.register("upper", upper)
spark.sql("SELECT upper('aBcD')").show()

+-----------+
|upper(aBcD)|
+-----------+
|       ABCD|
+-----------+



In [0]:
# DataFrame 기반 SQL에 적용
sdf.createOrReplaceTempView("sdf")
spark.sql("""SELECT name, upper(name) as `Capital Name` FROM sdf""").show()


+-------------+-------------+
|         name| Capital Name|
+-------------+-------------+
| luka doncici| LUKA DONCICI|
| lebron james| LEBRON JAMES|
|austin reaves|AUSTIN REAVES|
+-------------+-------------+



In [0]:
sdf_v2 = sdf.withColumn("Capital Name", expr("upper(name)"))
sdf_v2.show()

sdf_v3 = sdf.withColumn("Captial Name", upperUDF("name"))
sdf_v3.show()


+---+-------------+------+-------------+
| id|         name|salary| Capital Name|
+---+-------------+------+-------------+
|  1| luka doncici|   100| LUKA DONCICI|
|  2| lebron james|   200| LEBRON JAMES|
|  3|austin reaves|   300|AUSTIN REAVES|
+---+-------------+------+-------------+

+---+-------------+------+-------------+
| id|         name|salary| Captial Name|
+---+-------------+------+-------------+
|  1| luka doncici|   100| LUKA DONCICI|
|  2| lebron james|   200| LEBRON JAMES|
|  3|austin reaves|   300|AUSTIN REAVES|
+---+-------------+------+-------------+



In [0]:
# 예제 3: pyspark.sql.functions.pandas_udf
# 가장 성능이 좋은데 내부적으로 Partition 단위로 Pandas 엔진을 사용
# 그래서 입력과 출력이 모두 판다스 시리즈이어야함
from pyspark.sql.functions import pandas_udf

In [0]:
@pandas_udf(StringType())
def upper_udf2(s: pd.Series) -> pd.Series:
   return s.str.upper()

upperUDF = spark.udf.register("upper_udf", upper_udf2)
spark.sql('SELECT *, upper_udf(name) `Capital Name` FROM sdf').show()
sdf.select("id", "name", upperUDF("name").alias("Capital Name")).show()

+---+-------------+------+-------------+
| id|         name|salary| Capital Name|
+---+-------------+------+-------------+
|  1| luka doncici|   100| LUKA DONCICI|
|  2| lebron james|   200| LEBRON JAMES|
|  3|austin reaves|   300|AUSTIN REAVES|
+---+-------------+------+-------------+

+---+-------------+-------------+
| id|         name| Capital Name|
+---+-------------+-------------+
|  1| luka doncici| LUKA DONCICI|
|  2| lebron james| LEBRON JAMES|
|  3|austin reaves|AUSTIN REAVES|
+---+-------------+-------------+



In [0]:
@pandas_udf(FloatType())
def multiply_110percent(v: pd.Series) -> pd.Series:
   return v*1.1

multiply110percentUDF = spark.udf.register('multiply_110percent', multiply_110percent)
spark.sql('SELECT *, multiply_110percent(salary) `New Salary` FROM sdf').show()
sdf.select("id", "name", "salary", multiply110percentUDF("salary").alias("New Salary")).show()

+---+-------------+------+----------+
| id|         name|salary|New Salary|
+---+-------------+------+----------+
|  1| luka doncici|   100|     110.0|
|  2| lebron james|   200|     220.0|
|  3|austin reaves|   300|     330.0|
+---+-------------+------+----------+

+---+-------------+------+----------+
| id|         name|salary|New Salary|
+---+-------------+------+----------+
|  1| luka doncici|   100|     110.0|
|  2| lebron james|   200|     220.0|
|  3|austin reaves|   300|     330.0|
+---+-------------+------+----------+



# 본격 학습

## 테스트 데이터 생성

In [0]:
# 샘플 데이터 생성
data = {
    'name': ['김철수', '이영희', '박민준', '최지연', '정현우'],
    'age': [25, 30, 22, 35, 28],
    'salary': [50000, 60000, 45000, 70000, 55000]
}

# Pandas 데이터프레임 생성
pdf = pd.DataFrame(data)
print("===== Pandas 데이터프레임 =====")
print(pdf)

# Spark 데이터프레임 생성
sdf = spark.createDataFrame(pdf)
print("\n===== Spark 데이터프레임 =====")
sdf.show()

===== Pandas 데이터프레임 =====
  name  age  salary
0  김철수   25   50000
1  이영희   30   60000
2  박민준   22   45000
3  최지연   35   70000
4  정현우   28   55000

===== Spark 데이터프레임 =====
+------+---+------+
|  name|age|salary|
+------+---+------+
|김철수| 25| 50000|
|이영희| 30| 60000|
|박민준| 22| 45000|
|최지연| 35| 70000|
|정현우| 28| 55000|
+------+---+------+



## 예제 1: 간단한 데이터프레임 조작 및 apply/lambda 함수 사용
급여 일괄 10% 인상 계산

In [0]:
# Pandas: apply/lambda를 사용하여 급여 10% 인상
# 기본값은 axis=0으로 열 방향
print("Pandas로 구현:")
pdf['increased_salary'] = pdf['salary'].apply(lambda x: x * 1.1)
pdf.head(5)

Pandas로 구현:


,name,age,salary,increased_salary
0,김철수,25,50000,55000.0
1,이영희,30,60000,66000.0
2,박민준,22,45000,49500.0
3,최지연,35,70000,77000.0
4,정현우,28,55000,60500.0


In [0]:
def multiply_110percent(x):
    return x*1.1

pdf['increased_salary_v2'] = pdf['salary'].apply(multiply_110percent)
pdf.head(5)

,name,age,salary,increased_salary,increased_salary_v2
0,김철수,25,50000,55000.0,55000.0
1,이영희,30,60000,66000.0,66000.0
2,박민준,22,45000,49500.0,49500.0
3,최지연,35,70000,77000.0,77000.0
4,정현우,28,55000,60500.0,60500.0


In [0]:
pdf[["age", "salary"]].apply(lambda col: col.max() - col.min()).reset_index(name="range")

,index,range
0,age,13
1,salary,25000


In [0]:
from pyspark.sql.functions import *

# Spark: col() 함수와 수식을 사용
print("\nSpark로 구현:")
sdf = sdf.withColumn('increased_salary', col('salary') * 1.1)
sdf.show()


Spark로 구현:
+------+---+------+-----------------+
|  name|age|salary| increased_salary|
+------+---+------+-----------------+
|김철수| 25| 50000|55000.00000000001|
|이영희| 30| 60000|          66000.0|
|박민준| 22| 45000|49500.00000000001|
|최지연| 35| 70000|          77000.0|
|정현우| 28| 55000|60500.00000000001|
+------+---+------+-----------------+



In [0]:
@pandas_udf(FloatType())
def multiply_110percent(v: pd.Series) -> pd.Series:
   return v*1.1

sdf.createOrReplaceTempView("sdf")
multiply110percentUDF = spark.udf.register('multiply_110percent', multiply_110percent)
spark.sql('SELECT name, age, salary, multiply_110percent(salary) `increased_salary` FROM sdf').show()
sdf.select("name", "age", "salary", multiply110percentUDF("salary").alias("increased_salary")).show()

+------+---+------+----------------+
|  name|age|salary|increased_salary|
+------+---+------+----------------+
|김철수| 25| 50000|         55000.0|
|이영희| 30| 60000|         66000.0|
|박민준| 22| 45000|         49500.0|
|최지연| 35| 70000|         77000.0|
|정현우| 28| 55000|         60500.0|
+------+---+------+----------------+

+------+---+------+----------------+
|  name|age|salary|increased_salary|
+------+---+------+----------------+
|김철수| 25| 50000|         55000.0|
|이영희| 30| 60000|         66000.0|
|박민준| 22| 45000|         49500.0|
|최지연| 35| 70000|         77000.0|
|정현우| 28| 55000|         60500.0|
+------+---+------+----------------+



In [0]:

spark.sql("""
    SELECT
        (MAX(age)-MIN(age)) as age,
        (MAX(salary)-MIN(salary)) as salary
FROM sdf
""").show()

+---+------+
|age|salary|
+---+------+
| 13| 25000|
+---+------+



In [0]:
spark.sql("""
    SELECT
        'age' as index,
        (MAX(age)-MIN(age)) as range
    FROM sdf

    UNION

    SELECT
        'salary',
        (MAX(salary)-MIN(salary))
    FROM sdf
""").show()

+------+-----+
| index|range|
+------+-----+
|   age|   13|
|salary|25000|
+------+-----+



## 예제 2: 복잡한 로직을 적용한 컬럼 추가

나이 기반 카테고리 추가: 청년, 중년, 장년

In [0]:
# Pandas: apply/lambda를 사용하여 나이 기반 카테고리 추가
print("Pandas로 구현:")
def age_category(age):
    if age < 35:
        return "청년"
    elif age < 55:
        return "중년"
    else:
        return "장년"

pdf['age_category'] = pdf['age'].apply(age_category)
pdf.head()

Pandas로 구현:


,name,age,salary,increased_salary,increased_salary_v2,age_category
0,김철수,25,50000,55000.0,55000.0,청년
1,이영희,30,60000,66000.0,66000.0,청년
2,박민준,22,45000,49500.0,49500.0,청년
3,최지연,35,70000,77000.0,77000.0,중년
4,정현우,28,55000,60500.0,60500.0,청년


In [0]:
# Spark: 일반 UDF(User Defined Function) 사용
print("\nSpark로 구현:")
@udf(StringType())
def age_category_udf(age):
    if age < 35:
        return "청년"
    elif age < 55:
        return "중년"
    else:
        return "장년"

sdf = sdf.withColumn('age_category', age_category_udf(col('age')))
sdf = sdf.withColumn('age_category_v2', age_category_udf(sdf.age))
sdf = sdf.withColumn('age_category_v3', age_category_udf(expr("age")))

sdf.show()


Spark로 구현:
+------+---+------+-----------------+------------+---------------+---------------+
|  name|age|salary| increased_salary|age_category|age_category_v2|age_category_v3|
+------+---+------+-----------------+------------+---------------+---------------+
|김철수| 25| 50000|55000.00000000001|        청년|           청년|           청년|
|이영희| 30| 60000|          66000.0|        청년|           청년|           청년|
|박민준| 22| 45000|49500.00000000001|        청년|           청년|           청년|
|최지연| 35| 70000|          77000.0|        중년|           중년|           중년|
|정현우| 28| 55000|60500.00000000001|        청년|           청년|           청년|
+------+---+------+-----------------+------------+---------------+---------------+



In [0]:
# Spark: Pandas UDF(User Defined Function) 사용
@pandas_udf(StringType())  
def age_category_pandas_udf(age_series: pd.Series) -> pd.Series:
    return age_series.apply(lambda age: "청년" if age < 35 else "중년" if age < 55 else "장년")


sdf = sdf.withColumn('age_category', age_category_pandas_udf(col('age')))
sdf = sdf.withColumn('age_category_v2', age_category_pandas_udf(sdf.age))
sdf = sdf.withColumn('age_category_v3', age_category_pandas_udf(expr("age")))

sdf.show()

+------+---+------+-----------------+------------+---------------+---------------+
|  name|age|salary| increased_salary|age_category|age_category_v2|age_category_v3|
+------+---+------+-----------------+------------+---------------+---------------+
|김철수| 25| 50000|55000.00000000001|        청년|           청년|           청년|
|이영희| 30| 60000|          66000.0|        청년|           청년|           청년|
|박민준| 22| 45000|49500.00000000001|        청년|           청년|           청년|
|최지연| 35| 70000|          77000.0|        중년|           중년|           중년|
|정현우| 28| 55000|60500.00000000001|        청년|           청년|           청년|
+------+---+------+-----------------+------------+---------------+---------------+

